In [1]:
"""
"""
from __future__ import print_function
import os
import argparse
import pickle
from datetime import datetime
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# tensorflow/keras imports 
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import l1
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy

# qkeras and pruning imports 
from qkeras.qlayers import QDense, QActivation
from qkeras import QBatchNormalization
from qkeras.quantizers import quantized_bits, quantized_relu
from qkeras.utils import _add_supported_quantized_objects
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
import tensorflow_model_optimization as tfmot

2024-06-04 20:55:15.302505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 20:55:15.560634: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-04 20:55:15.560664: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-06-04 20:55:16.303761: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

## Helper functions

In [2]:
def get_checkpoint_filename(checkpoint_dir, experiment_num, checkpoint_filename):
    return os.path.join(checkpoint_dir, f'{experiment_num}', checkpoint_filename)

def get_callbacks(checkpoint_exp_filename, prune):
    callbacks = [
            ModelCheckpoint(
            checkpoint_exp_filename,
            monitor="val_loss",
            verbose=0,
            save_best_only=True,
            save_weights_only=False,
            save_freq="epoch",
        ),
        ReduceLROnPlateau(patience=75, min_delta=1**-6),
    ]
    if prune:
        callbacks.append(pruning_callbacks.UpdatePruningStep())
    return callbacks

def load_checkpoint(filename):
    co = {}
    _add_supported_quantized_objects(co)
    model = load_model(filename, custom_objects=co, compile=False)
    return model

def one_hot_encode(data):
    y_encoded = np.zeros([data.shape[0],2], dtype=np.int32)
    for idx, x in enumerate(data):
        if x == 1:
            y_encoded[idx][1] = 1
        else:
            y_encoded[idx][0] = 1
    return y_encoded

In [3]:
def load_data(data_dir, window_start, window_end, bit_shift):
    print('#########################################################')
    print(f'Loading data from dir: {data_dir}')
    print('\tWindow start:', window_start)
    print('\tWindow end:', window_end)
    print('#########################################################')
    X_train_val = np.load(os.path.join(data_dir, f'X_train_{window_start}_{window_end}.npy'))
    X_test = np.load(os.path.join(data_dir, f'X_test_{window_start}_{window_end}.npy'))    
    y_train_val = np.load(os.path.join(data_dir, f'y_train.npy'))
    y_test = np.load(os.path.join(data_dir, f'y_test.npy'))

    y_train_val = one_hot_encode(y_train_val)
    y_test = one_hot_encode(y_test)

    if bit_shift:
        shift = 3
        print(X_train_val.max())
        X_train_val = np.right_shift(X_train_val.astype(np.int32), shift)
        X_test = np.right_shift(X_test.astype(np.int32), shift)
        print(X_train_val.max())

    return X_train_val, X_test, y_train_val, y_test


#########################################################
# Model
#########################################################
def build_model(model_type, input_shape, quantize, is_pruning):
    model = Sequential()
    # build model 
    if model_type == 'mlp':
        # csr = range(0, 770)
        # sr = len(csr)
        # hn = sr*2 * 1
        sr = int(input_shape/2)
        hn = sr * 2
        if quantize:
            model.add(QDense(int(hn/8), input_shape=(sr*2,), name='fc1', kernel_quantizer=quantized_bits(6,0,alpha=1), bias_quantizer=quantized_bits(6,0,alpha=1),))
            model.add(Activation('relu'))
            model.add(BatchNormalization())
            model.add(QDense(2, name='fc2', kernel_quantizer=quantized_bits(6,0,alpha=1), bias_quantizer=quantized_bits(6,0,alpha=1)))
            model.add(Activation('relu'))
        else:
            model.add(Dense(int(hn/8), activation='relu', input_shape=(sr*2,)))
            model.add(BatchNormalization())
            model.add(Dense(2, activation='relu'))
    elif model_type == 'single':
        if quantize: 
            model.add(QDense(2, input_shape=(input_shape,), name='fc1', kernel_quantizer=quantized_bits(6,0,alpha=1), bias_quantizer=quantized_bits(6,0,alpha=1),))
            model.add(BatchNormalization())
        else:
            model.add(Dense(2, input_shape=(input_shape,), name='fc1'))
            model.add(BatchNormalization())
    # pruning options 
    if is_pruning == True:
        pruning_params = {"pruning_schedule": pruning_schedule.ConstantSparsity(0.85, begin_step=200, frequency=100)}
        # Define model for pruning
        pruning_params = {'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50, final_sparsity=0.80, begin_step=200, end_step=1000)}
        model = prune.prune_low_magnitude(model, **pruning_params)
    return model 


In [4]:
#########################################################
# Training 
#########################################################
def train(model, X_train_val, y_train_val, lr, batch_size, epochs, val_split, callbacks):
    initial_learning_rate = lr
    opt = Adam(learning_rate=initial_learning_rate)
    model.compile(optimizer=opt, loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    history = model.fit(X_train_val, 
              y_train_val, 
              batch_size=batch_size,
              epochs=epochs, 
              validation_split=val_split, 
              shuffle=True, 
              callbacks=callbacks
    )
    return history



In [5]:
def log_metrics(model, model_type, window_start, window_end, experiment_number, quantize, prune, train_acc, test_acc):
    # model parameters and sparsity 
    model_params_count = model.count_params()
    zero_params_count = 0
    for layer in model.layers:
        if len(layer.get_weights()) > 0:  # Some layers may not have weights
            params = layer.get_weights()
            for param in params:
                zero_params_count += np.sum(param == 0)

    print(f'Number of parameters: {model_params_count}')
    print(f'Number of zero parameters: {zero_params_count}')

    # save to file 
    with open('notebook_dse.txt', 'a') as file:
        quantize = 1 if quantize == True else 0
        prune = 1 if prune == True else 0
        run_results = f'{model_type}_{window_start}_{window_end}, {experiment_number}, {quantize}, {prune}, {window_start}, {window_end}, {train_acc}, {test_acc}, {model_params_count}, {zero_params_count}\n'
        file.write(run_results)

def evaluate_model(model, checkpoint_exp_filename, history, X_train, y_train, X_test, y_test):
    if prune == True:
        model = strip_pruning(model)

    print(f'Saved checkpoint to: {checkpoint_exp_filename}')
    history_file = checkpoint_exp_filename.replace(".h5", "-history.pkl")
    print(f'Saving history to: {history_file}')
    with open(history_file, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

    # get test accuracy
    model = load_checkpoint(checkpoint_exp_filename)
    y_pred = model.predict(X_test)
    
    test_acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
    print('Keras Test Accuracy: {}'.format(test_acc))

    # get train accuracy
    y_pred = model.predict(X_train)
    train_acc = accuracy_score(np.argmax(y_train, axis=1), np.argmax(y_pred, axis=1))
    print('Keras Train Accuracy: {}'.format(train_acc))

    return train_acc, test_acc
    

# Training parameters

In [6]:
# data options 
data_dir = '../data/malab_05272024/npz'
batch_size = 12800
window_start = 0
window_end = 770
bitshift_data = False
# training options 
max_epochs = 50
val_split = 0.03
lr = 1e-2
# model options 
model_type = 'single'   # options: [mlp, single]
quantize = False
prune = False
# saving options 
checkpoint_dir = '../checkpoints'
checkpoint_filename = 'model_best.h5'
experiment_number = 0
# space exploration parameters 
model_types = ['single', 'mlp']
num_experiments = 3
quantize = True
prune = True
end_windwows = list(range(150, 751, 50))   # [150, 200, 250... 750]
end_windwows.append(770)
start_windows = [100] * len(end_windwows)  # [100, 100, 100... 100]
start_windows.append(0)

In [7]:
for model_type in model_types:
    for start, end in zip(start_windows, end_windwows):
        # load data 
        X_train, X_test, y_train, y_test = load_data(data_dir, start, end, bitshift_data)
        for exp_number in range(num_experiments):
            # build model 
            # model = build_model(model_type, int((end-start)*2), quantize=False, is_pruning=False)
            # model.build((batch_size, int((end-start)*2)))
            model = build_model(model_type, X_train.shape[1], quantize=False, is_pruning=False)
            model.build((batch_size, X_train.shape[1]))

            # train 
            checkpoint_exp_filename = get_checkpoint_filename(checkpoint_dir, experiment_number, checkpoint_filename)
            callbacks = get_callbacks(checkpoint_exp_filename, prune)
            history = train(model, X_train, y_train, lr, batch_size, max_epochs, val_split, callbacks)
            # eval 
            train_acc, test_acc = evaluate_model(model, checkpoint_exp_filename, history, X_train, y_train, X_test, y_test)
            log_metrics(model, model_type, start, end, exp_number, False, False, train_acc, test_acc)

        if quantize:
            for exp_number in range(num_experiments):
                # build model 
                model = build_model(model_type, X_train.shape[1], quantize=True, is_pruning=False)
                model.build((batch_size, X_train.shape[1]))
                # train 
                checkpoint_exp_filename = get_checkpoint_filename(checkpoint_dir, experiment_number, checkpoint_filename)
                callbacks = get_callbacks(checkpoint_exp_filename, prune)
                history = train(model, X_train, y_train, lr, batch_size, max_epochs, val_split, callbacks)
                # eval 
                train_acc, test_acc = evaluate_model(model, checkpoint_exp_filename, history, X_train, y_train, X_test, y_test)
                log_metrics(model, model_type, start, end, exp_number, True, False, train_acc, test_acc)

#########################################################
Loading data from dir: ../data/malab_05272024/npz
	Window start: 0
	Window end: 770
#########################################################


2024-06-04 20:57:34.425308: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-06-04 20:57:34.425521: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-06-04 20:57:34.425548: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (correlator4.fnal.gov): /proc/driver/nvidia/version does not exist
2024-06-04 20:57:34.425819: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 20:57:37.011400: W tensorflow/tsl/framework/

Epoch 1/50
69/69 [==============================] - 2s 24ms/step - loss: 0.3225 - accuracy: 0.9036 - val_loss: 0.3586 - val_accuracy: 0.9080 - lr: 0.0100
Epoch 2/50
69/69 [==============================] - 1s 19ms/step - loss: 0.3053 - accuracy: 0.9126 - val_loss: 0.3197 - val_accuracy: 0.9099 - lr: 0.0100
Epoch 3/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3048 - accuracy: 0.9135 - val_loss: 0.3139 - val_accuracy: 0.9105 - lr: 0.0100
Epoch 4/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3046 - accuracy: 0.9138 - val_loss: 0.3127 - val_accuracy: 0.9106 - lr: 0.0100
Epoch 5/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3047 - accuracy: 0.9139 - val_loss: 0.3129 - val_accuracy: 0.9104 - lr: 0.0100
Epoch 6/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3046 - accuracy: 0.9140 - val_loss: 0.3128 - val_accuracy: 0.9107 - lr: 0.0100
Epoch 7/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3

2024-06-04 20:58:56.973300: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 5544000000 exceeds 10% of free system memory.


28125/28125 [==============================] - 18s 646us/step
Keras Train Accuracy: 0.91403
Number of parameters: 3090
Number of zero parameters: 0


...

Epoch 1/50
69/69 [==============================] - 2s 24ms/step - loss: 0.3264 - accuracy: 0.9017 - val_loss: 0.3576 - val_accuracy: 0.9076 - lr: 0.0100
Epoch 2/50
69/69 [==============================] - 1s 19ms/step - loss: 0.3054 - accuracy: 0.9124 - val_loss: 0.3195 - val_accuracy: 0.9095 - lr: 0.0100
Epoch 3/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3048 - accuracy: 0.9134 - val_loss: 0.3147 - val_accuracy: 0.9104 - lr: 0.0100
Epoch 4/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3046 - accuracy: 0.9138 - val_loss: 0.3133 - val_accuracy: 0.9107 - lr: 0.0100
Epoch 5/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3046 - accuracy: 0.9139 - val_loss: 0.3134 - val_accuracy: 0.9105 - lr: 0.0100
Epoch 6/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3046 - accuracy: 0.9139 - val_loss: 0.3123 - val_accuracy: 0.9104 - lr: 0.0100
Epoch 7/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3

2024-06-04 21:00:49.874056: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 5544000000 exceeds 10% of free system memory.


28125/28125 [==============================] - 17s 609us/step
Keras Train Accuracy: 0.9141011111111111
Number of parameters: 3090
Number of zero parameters: 0


2024-06-04 21:01:18.117402: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 5377680000 exceeds 10% of free system memory.


Epoch 1/50
69/69 [==============================] - 2s 25ms/step - loss: 0.3234 - accuracy: 0.9032 - val_loss: 0.3597 - val_accuracy: 0.9073 - lr: 0.0100
Epoch 2/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3053 - accuracy: 0.9125 - val_loss: 0.3203 - val_accuracy: 0.9099 - lr: 0.0100
Epoch 3/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3048 - accuracy: 0.9135 - val_loss: 0.3149 - val_accuracy: 0.9101 - lr: 0.0100
Epoch 4/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3046 - accuracy: 0.9139 - val_loss: 0.3137 - val_accuracy: 0.9106 - lr: 0.0100
Epoch 5/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3046 - accuracy: 0.9139 - val_loss: 0.3125 - val_accuracy: 0.9106 - lr: 0.0100
Epoch 6/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3046 - accuracy: 0.9139 - val_loss: 0.3124 - val_accuracy: 0.9105 - lr: 0.0100
Epoch 7/50
69/69 [==============================] - 1s 20ms/step - loss: 0.3